In [2]:
import numpy as np
import pandas as pd 
from scipy import stats
from scipy.spatial import distance

# How to compute Mahalanobis Distance in Python

In [3]:
import pandas as pd
import scipy as sp
import numpy as np

df = pd.read_csv('diamonds.csv').iloc[:, [0,4,6]]
df.head()

,carat,depth,price
0,0.23,61.5,326
1,0.21,59.8,326
2,0.23,56.9,327
3,0.29,62.4,334
4,0.31,63.3,335


# Function to calculate Mahalanobis Distance

In [4]:
def mahalanobis(x=None, data=None, cov=None):
    """Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    x_minus_mu = x - np.mean(data)
    if not cov:
        cov = np.cov(data.values.T)
    inv_covmat = sp.linalg.inv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()

df_x = df[['carat', 'depth', 'price']].head(500)
df_x['mahala'] = mahalanobis(x=df_x, data=df[['carat', 'depth', 'price']])
df_x.head()

,carat,depth,price,mahala
0,0.23,61.5,326,5.239696e+07
1,0.21,59.8,326,5.239896e+07
2,0.23,56.9,327,5.239813e+07
3,0.29,62.4,334,5.239210e+07
4,0.31,63.3,335,5.239031e+07


# Usecase 1: Multivariate outlier detection using Mahalanobis distance

In [5]:
# Critical values for two degrees of freedom
from scipy.stats import chi2
chi2.ppf((1-0.01), df=2)

9.21034037197618

That mean an observation can be considered as extreme if its Mahalanobis distance exceeds 9.21. If you prefer P values instead to determine if an observation is extreme or not, the P values can be computed as follows:

In [6]:
# Compute the P-Values
df_x['p_value'] = 1 - chi2.cdf(df_x['mahala'], 2)

# Extreme values with a significance level of 0.01
df_x.loc[df_x.p_value < 0.01].head(10)

,carat,depth,price,mahala,p_value
0,0.23,61.5,326,5.239696e+07,0.0
1,0.21,59.8,326,5.239896e+07,0.0
2,0.23,56.9,327,5.239813e+07,0.0
3,0.29,62.4,334,5.239210e+07,0.0
4,0.31,63.3,335,5.239031e+07,0.0
5,0.24,62.8,336,5.239594e+07,0.0
6,0.24,62.3,336,5.239606e+07,0.0
7,0.26,61.9,337,5.239460e+07,0.0
8,0.22,65.1,337,5.239695e+07,0.0
9,0.23,59.4,338,5.239759e+07,0.0


If you compare the above observations against rest of the dataset, they are clearly extreme.

# Usecase 2: Mahalanobis Distance for Classification Problems
Mahalanobis distance can be used for classification problems. A naive implementation of a Mahalanobis classifier is coded below. The intuition is that, an observation is assigned the class that it is closest to based on the Mahalanobis distance. Let’s see an example implementation on the BreastCancer dataset, where the objective is to determine if a tumour is benign or malignant

In [16]:
df2 = pd.read_csv('BreastCancer.csv', usecols=['Cl.thickness', 'Cell.size', 'Marg.adhesion', 'Epith.c.size', 'Bare.nuclei', 'Bl.cromatin', 'Normal.nucleoli', 'Mitoses', 'Class'])
df2.dropna(inplace=True)  # drop missing values.
df2.head()

,Cl.thickness,Cell.size,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
0,5,1,1,2,1.0,3,1,1,0
1,5,4,5,7,10.0,3,2,1,0
2,3,1,1,2,2.0,3,1,1,0
3,6,8,1,3,4.0,3,7,1,0
4,4,1,3,2,1.0,3,1,1,0


Let’s split the dataset in 70:30 ratio as Train and Test. And the training dataset is split into homogeneous groups of ‘pos'(1) and ‘neg'(0) classes. To predict the class of the test dataset, we measure the Mahalanobis distances between a given observation (row) and both the positive (xtrain_pos) and negative datasets(xtrain_neg). Then that observation is assigned the class based on the group it is closest to.

In [17]:
df2.columns

Index(['Cl.thickness', 'Cell.size', 'Marg.adhesion', 'Epith.c.size',
       'Bare.nuclei', 'Bl.cromatin', 'Normal.nucleoli', 'Mitoses', 'Class'],
      dtype='object')

In [20]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(df2.drop('Class', axis=1), df2['Class'], test_size=.3, random_state=100)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=100)

In [21]:
# Split the training data as pos and neg
xtrain_pos = xtrain.loc[ytrain == 1, :]
xtrain_neg = xtrain.loc[ytrain == 0, :]

Let’s build the MahalanobiBinaryClassifier. To do that, you need to define the predict_proba() and the predict() methods. This classifier does not require a separate fit() (training) method.

In [22]:
class MahalanobisBinaryClassifier():
    def __init__(self, xtrain, ytrain):
        self.xtrain_pos = xtrain.loc[ytrain == 1, :]
        self.xtrain_neg = xtrain.loc[ytrain == 0, :]

    def predict_proba(self, xtest):
        pos_neg_dists = [(p,n) for p, n in zip(mahalanobis(xtest, self.xtrain_pos), mahalanobis(xtest, self.xtrain_neg))]
        return np.array([(1-n/(p+n), 1-p/(p+n)) for p,n in pos_neg_dists])

    def predict(self, xtest):
        return np.array([np.argmax(row) for row in self.predict_proba(xtest)])


clf = MahalanobisBinaryClassifier(xtrain, ytrain)        
pred_probs = clf.predict_proba(xtest)
pred_class = clf.predict(xtest)

# Pred and Truth
pred_actuals = pd.DataFrame([(pred, act) for pred, act in zip(pred_class, ytest)], columns=['pred', 'true'])
print(pred_actuals[:5])

   pred  true
0     1     0
1     1     1
2     0     0
3     1     0
4     0     0


Let’s see how the classifier performed on the test dataset.

In [23]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix
truth = pred_actuals.loc[:, 'true']
pred = pred_actuals.loc[:, 'pred']
scores = np.array(pred_probs)[:, 1]
print('AUROC: ', roc_auc_score(truth, scores))
print('\nConfusion Matrix: \n', confusion_matrix(truth, pred))
print('\nAccuracy Score: ', accuracy_score(truth, pred))
print('\nClassification Report: \n', classification_report(truth, pred))

AUROC:  0.9867692307692308

Confusion Matrix: 
 [[96 34]
 [ 0 75]]

Accuracy Score:  0.8341463414634146

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.74      0.85       130
           1       0.69      1.00      0.82        75

    accuracy                           0.83       205
   macro avg       0.84      0.87      0.83       205
weighted avg       0.89      0.83      0.84       205



# Usecase 3: One-Class Classification
One Class classification is a type of algorithm where the training dataset contains observations belonging to only one class.

With only that information known, the objective is to figure out if a given observation in a new (or test) dataset belongs to that class. You might wonder when would such a situation occur.

One Class classification is a type of algorithm where the training dataset contains observations belonging to only one class.

With only that information known, the objective is to figure out if a given observation in a new (or test) dataset belongs to that class. You might wonder when would such a situation occur.

In [24]:
df = pd.read_csv('BreastCancer.csv', usecols=['Cl.thickness', 'Cell.size', 'Marg.adhesion', 'Epith.c.size', 'Bare.nuclei', 'Bl.cromatin', 'Normal.nucleoli', 'Mitoses', 'Class'])
df.dropna(inplace=True)  # drop missing values.

Splitting 50% of the dataset into training and test. Only the 1’s are retained in the training data.

In [25]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(df.drop('Class', axis=1), df['Class'], test_size=.5, random_state=100)

# Split the training data as pos and neg
xtrain_pos = xtrain.loc[ytrain == 1, :]

Let’s build the MahalanobisOneClassClassifier and get the mahalanobis distance of each datapoint in x from the training set (xtrain_pos):

In [26]:
class MahalanobisOneclassClassifier():
    def __init__(self, xtrain, significance_level=0.01):
        self.xtrain = xtrain
        self.critical_value = chi2.ppf((1-significance_level), df=xtrain.shape[1]-1)
        print('Critical value is: ', self.critical_value)

    def predict_proba(self, xtest):
        mahalanobis_dist = mahalanobis(xtest, self.xtrain)
        self.pvalues = 1 - chi2.cdf(mahalanobis_dist, 2)
        return mahalanobis_dist

    def predict(self, xtest):
        return np.array([int(i) for i in self.predict_proba(xtest) > self.critical_value])

clf = MahalanobisOneclassClassifier(xtrain_pos, significance_level=0.05)
mahalanobis_dist = clf.predict_proba(xtest)

# Pred and Truth
mdist_actuals = pd.DataFrame([(m, act) for m, act in zip(mahalanobis_dist, ytest)], columns=['mahal', 'true_class'])
print(mdist_actuals[:5]) 

Critical value is:  14.067140449340167
       mahal  true_class
0  11.306536           0
1  12.824249           1
2  11.654914           0
3  11.298144           0
4  12.716892           0


We have the Mahalanobis distance and the actual class of each observation. I would expect those observations with low Mahalanobis distance to be 1’s. So, I sort the mdist_actuals by Mahalanobis distance and quantile cut the rows into 10 equal sized groups. The observations in the top quantiles should have more 1’s compared to the ones in the bottom. Let’s see

In [27]:
# quantile cut in 10 pieces
mdist_actuals['quantile'] = pd.qcut(mdist_actuals['mahal'], 
                                    q=[0, .10, .20, .3, .4, .5, .6, .7, .8, .9, 1], 
                                    labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# sort by mahalanobis distance
mdist_actuals.sort_values('mahal', inplace=True)
perc_truths = mdist_actuals.groupby('quantile').agg({'mahal': np.mean, 'true_class': np.sum}).rename(columns={'mahal':'avg_mahaldist', 'true_class':'sum_of_trueclass'})
print(perc_truths)

          avg_mahaldist  sum_of_trueclass
quantile                                 
1              6.191630                32
2              9.191590                28
3             10.790403                19
4             11.609966                10
5             12.434227                 5
6             13.181877                11
7             14.044480                 1
8             15.305396                 7
9             16.810820                 2
10            19.149191                15


If you notice above, nearly 90% of the 1’s (malignant cases) fall within the first 40%ile of the Mahalanobis distance.

Incidentally, all of these are lower than the critical value pf 14.05. So, let’s the critical value as the cutoff and mark those observations with Mahalanobis distance less than the cutoff as positive.

In [28]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix

# Positive if mahalanobis 
pred_actuals = pd.DataFrame([(int(p), y) for y, p in zip(ytest, clf.predict_proba(xtest) < clf.critical_value)], columns=['pred', 'true'])

# Accuracy Metrics
truth = pred_actuals.loc[:, 'true']
pred = pred_actuals.loc[:, 'pred']
print('\nConfusion Matrix: \n', confusion_matrix(truth, pred))
print('\nAccuracy Score: ', accuracy_score(truth, pred))
print('\nClassification Report: \n', classification_report(truth, pred))


Confusion Matrix: 
 [[ 87 125]
 [ 25 105]]

Accuracy Score:  0.5614035087719298

Classification Report: 
               precision    recall  f1-score   support

           0       0.78      0.41      0.54       212
           1       0.46      0.81      0.58       130

    accuracy                           0.56       342
   macro avg       0.62      0.61      0.56       342
weighted avg       0.66      0.56      0.55       342



So, without the knowledge of the benign class, we are able to accurately predict the class of 87% of the observations.

# Conclusion

In this post, we covered nearly everything about Mahalanobis distance: the intuition behind the formula, the actual calculation in python and how it can be used for multivariate anomaly detection, binary classification, and one-class classification. It is known to perform really well when you have a highly imbalanced dataset. Hope it was useful? Please leave your comments below and I will see you in the next one.  